In [ ]:
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir='/content/drive/MyDrive/train_cancer'
test_dir='/content/drive/MyDrive/test_cancer'
valid_dir='/content/drive/MyDrive/valid_cancer'
transform=transforms.Compose([ transforms.RandomResizedCrop(224)
                              ,transforms.ToTensor()
                              ,transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
data_tr=datasets.ImageFolder(train_dir,transform)
trian=DataLoader(data_tr,batch_size=15)

data_v=datasets.ImageFolder(valid_dir,transform)
valid=DataLoader(data_v,batch_size=15)

data_t=datasets.ImageFolder(test_dir,transform)
testm=DataLoader(data_t,batch_size=15)


In [ ]:
model_transfer = models.vgg16(pretrained=True)
for parm in model_transfer.features.parameters():
    parm.requires_grad = False
lastLayer=nn.Linear(model_transfer.classifier[6].in_features,3)
model_transfer.classifier[6]=lastLayer



Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
def train_m(train_loader,val_loader,n_ep,model,optmizer,lossF,savepath):
  min_val_loss=np.Inf

  for ep in range(1,n_ep):
    train_loss=0
    valid_loss=0
    model.train()
    for batch_indx ,(data,target) in enumerate(train_loader):
      if use_cuda:
        data, target = data.cuda(), target.cuda()
      optmizer.zero_grad()
      output=model(data)
      loss=lossF(output,target)
      loss.backward()
      optmizer.step()
      train_loss = train_loss + ((1 / (batch_indx + 1)) * (loss.data.item() - train_loss))

    model.eval() 
    for batch_indx ,(data,target) in enumerate(val_loader):
      if use_cuda:
        data, target = data.cuda(), target.cuda()
      output=model(data)
      loss=lossF(output,target)
      valid_loss = valid_loss + ((1 / (batch_indx + 1)) * (loss.data.item() - valid_loss))
      
    print('epoc {} \ttrining loss {:.6f} \tvaldition loss {:.6f}'.format(ep,train_loss,valid_loss))

    if(min_val_loss>valid_loss):
      torch.save(model.state_dict(),savepath)
      min_val_loss=valid_loss
      print('saved the model')

  return model

In [ ]:

use_cuda=torch.cuda.is_available()
if use_cuda:
  model_transfer=model_transfer.cuda()
lr=0.001
opmizer= optim.SGD(model_transfer.parameters(),lr)
lossF=nn.CrossEntropyLoss()
model=train_m(trian,valid,15,model_transfer,opmizer,lossF,'/content/bestModel')

epoc 1 	trining loss 0.061907 	valdition loss 0.010230
saved the model
epoc 2 	trining loss 0.007011 	valdition loss 0.004590
saved the model
epoc 3 	trining loss 0.004286 	valdition loss 0.002648
saved the model
epoc 4 	trining loss 0.002974 	valdition loss 0.001744
saved the model
epoc 5 	trining loss 0.002021 	valdition loss 0.002326
epoc 6 	trining loss 0.002022 	valdition loss 0.001306
saved the model
epoc 7 	trining loss 0.001606 	valdition loss 0.001442
epoc 8 	trining loss 0.001497 	valdition loss 0.001098
saved the model
epoc 9 	trining loss 0.001159 	valdition loss 0.000823
saved the model
epoc 10 	trining loss 0.000925 	valdition loss 0.000748
saved the model
epoc 11 	trining loss 0.000958 	valdition loss 0.000737
saved the model
epoc 12 	trining loss 0.000823 	valdition loss 0.000783
epoc 13 	trining loss 0.000747 	valdition loss 0.000494
saved the model
epoc 14 	trining loss 0.000832 	valdition loss 0.000563


In [ ]:
def test_model(test_m, model, criterion, use_cuda):

   
    test_loss = 0.
    correct = 0.
    total = 0.

   
    model.eval()

    for batch_idx, (data, target) in enumerate(test_m):
     
        if use_cuda:
            data, target = data.cuda(), target.cuda()
            model.cuda()
       
        output = model(data)
      
        loss = criterion(output, target)
         
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - test_loss))
      
        pred = output.data.max(1, keepdim=True)[1]
      
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
test_model(testm, model_transfer, lossF, use_cuda)

Test Loss: 0.000681


Test Accuracy: 100% (600/600)
